In [3262]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import xgboost

import datetime
from sklearn import preprocessing
sns.set()

train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')

# train.info()

In [3263]:
from datetime import datetime
train['date'] = pd.to_datetime(train['date'],format='%m/%d/%Y')
# train['month'] = train['date'].dt.month
# train['dayofweek'] = train['date'].dt.dayofweek
# train['year'] = train['date'].dt.year
# train['day'] = train['date'].dt.day
# train['dayofyear'] = train['date'].dt.dayofyear
# train['weekofyear'] = train['date'].dt.weekofyear

train.drop(['date'], axis=1, inplace=True) 

In [3264]:
train.isnull().sum()

Id                     0
game_seq               0
season_end             0
season_game_seq        0
playoff                0
team_id                0
Elo                 9197
opp_team_id            0
opp_Elo             7006
win_equivalent     12263
bet_ratio              0
home_crowd             0
opp_crowd              0
total_crowd            0
game_result            0
dtype: int64

In [3265]:
# g = sns.FacetGrid(train, col='game_result')
# g.map(plt.hist, 'Elo', bins=20)

# g = sns.FacetGrid(train, col='game_result')
# g.map(plt.hist, 'opp_Elo', bins=20)

# g = sns.FacetGrid(train, col='game_result')
# g.map(plt.hist, 'total_crowd', bins=20)



In [3266]:
for index,row in train.iterrows():
    if(row['bet_ratio']<0.13 or row['bet_ratio']>0.97):
        train.drop(index,inplace=True)  
    elif(row['opp_Elo']<1200 and row['opp_Elo']>1750):
        train.drop(index,inplace=True)  
    elif(row['Elo']<1175 and row['Elo']>1780):
        train.drop(index,inplace=True)
    elif(row['win_equivalent']<14 or row['win_equivalent']>68):
        train.drop(index,inplace=True)   

In [3267]:
# train = pd.concat([train,pd.get_dummies(train['team_id'], prefix='team_id',dummy_na=False)],axis=1)
# train = pd.concat([train,pd.get_dummies(train['opp_team_id'], prefix='opp_team_id',dummy_na=False)],axis=1)

train['team_id'] = train['team_id'].str.extract('(\\d+)', expand=True)
train['opp_team_id'] = train['opp_team_id'].str.extract('(\\d+)', expand=True)

train['team_id']=train['team_id'].astype(int)
train['opp_team_id']=train['opp_team_id'].astype(int)

In [3268]:
train['bet_ratio'].describe()


count    44876.000000
mean         0.621678
std          0.177819
min          0.130111
25%          0.498947
50%          0.640104
75%          0.760705
max          0.969080
Name: bet_ratio, dtype: float64

In [3269]:
# train['Elo'].fillna(train.groupby('team_id')['Elo'].transform('median'), inplace=True)
# train['opp_Elo'].fillna(train.groupby('opp_team_id')['opp_Elo'].transform('median'), inplace=True)
# train['win_equivalent'].fillna(train.groupby('season_end')['win_equivalent'].transform('median'), inplace=True)

train["temp"] = train["team_id"].map(str) + train["season_end"].map(str)
train["temp1"] = train["opp_team_id"].map(str) + train["season_end"].map(str)

train['Elo'].fillna(train.groupby('temp')['Elo'].transform('median'), inplace=True)
train['opp_Elo'].fillna(train.groupby('temp1')['opp_Elo'].transform('median'), inplace=True)
train['win_equivalent'].fillna(train.groupby('season_end')['win_equivalent'].transform('median'), inplace=True)

train = train.dropna(axis = 0, how ='any')

train.drop(['temp'], axis=1, inplace=True) 
train.drop(['temp1'], axis=1, inplace=True) 


In [3270]:


# for index,row in train.iterrows():
    
#     if(train.loc[index,'bet_ratio']<0.5):
#         train.at[index,'Bet']=-1 
#     if(train.loc[index,'bet_ratio']>0.8):
#         train.at[index,'Bet']=1
#     if(train.loc[index,'bet_ratio']>=0.5 and train.loc[index,'bet_ratio']<=0.8):
#         train.at[index,'Bet']=0
#     if(train.loc[index,'Elo']<1350):
#         train.at[index,'elo1']=-1 
#     if(train.loc[index,'Elo']>1645):
#         train.at[index,'elo1']=1
#     if(train.loc[index,'Elo']>=1350 and train.loc[index,'Elo']<=1645):
#         train.at[index,'elo1']=0
#     if(train.loc[index,'opp_Elo']<1355):
#         train.at[index,'elo2']=-1 
#     if(train.loc[index,'opp_Elo']>1660):
#         train.at[index,'elo2']=1
#     if(train.loc[index,'opp_Elo']>=1355 and train.loc[index,'opp_Elo']<=1660):
#         train.at[index,'elo2']=0


In [3271]:
train['diff']=train.Elo-train.opp_Elo
train.head()


,Id,game_seq,season_end,season_game_seq,playoff,team_id,Elo,opp_team_id,opp_Elo,win_equivalent,bet_ratio,home_crowd,opp_crowd,total_crowd,game_result,diff
0,1,39323,1997,2,False,25,1511.6364,10,1535.13505,42.985456,0.616437,6600,4800,11400,1,-23.49865
1,2,59211,2012,87,True,22,1690.4612,42,1732.21640,59.721931,0.583044,5600,4300,9900,0,-41.75520
2,3,60691,2014,12,False,18,1550.2258,11,1612.84485,42.857415,0.606468,8600,3800,12400,0,-62.61905
3,4,31900,1990,69,False,12,1450.0503,17,1629.69850,39.305004,0.390694,7200,4000,11200,0,-179.64820
4,5,48134,2004,30,False,27,1372.9828,12,1467.52120,28.311192,0.492179,8600,4900,13500,0,-94.53840


In [3272]:


train.drop(['Id'], axis=1, inplace=True)
train.drop(['game_seq'], axis=1, inplace=True)  
# train.drop(['season_end'], axis=1, inplace=True)        
# train.drop(['season_game_seq'], axis=1, inplace=True) 
# train.drop(['playoff'], axis=1, inplace=True)
train.drop(['team_id'], axis=1, inplace=True) 
# train.drop(['Elo'], axis=1, inplace=True)
train.drop(['opp_team_id'], axis=1, inplace=True)
# train.drop(['opp_Elo'], axis=1, inplace=True) 
# train.drop(['win_equivalent'], axis=1, inplace=True) 
# train.drop(['bet_ratio'], axis=1, inplace=True) 
# train.drop(['home_crowd'], axis=1, inplace=True) 
train.drop(['opp_crowd'], axis=1, inplace=True) 
# train.drop(['total_crowd'], axis=1, inplace=True) 

In [3273]:
labelencoder = LabelEncoder()
train['playoff'] = labelencoder.fit_transform(train['playoff'])
# train['team_id'] = labelencoder.fit_transform(train['team_id'])
# train['opp_team_id'] = labelencoder.fit_transform(train['opp_team_id'])

In [3274]:
# train['diff'].unique()

In [3275]:
# train.head()

In [3276]:
train.corr()

,season_end,season_game_seq,playoff,Elo,opp_Elo,win_equivalent,bet_ratio,home_crowd,total_crowd,game_result,diff
season_end,1.000000,0.021931,-0.010427,0.062100,0.042107,0.004989,0.004106,-0.000007,0.000202,0.022822,0.014356
season_game_seq,0.021931,1.000000,0.445790,0.105837,0.104481,0.096641,-0.002174,0.003055,0.000224,-0.011366,0.000601
playoff,-0.010427,0.445790,1.000000,0.235446,0.220502,0.212934,0.026245,0.004817,0.005992,-0.021292,0.010037
Elo,0.062100,0.105837,0.235446,1.000000,0.055561,0.818734,0.681155,-0.007454,-0.001393,-0.269000,0.685243
opp_Elo,0.042107,0.104481,0.220502,0.055561,1.000000,0.020775,-0.663164,0.002475,0.005301,0.236265,-0.689116
win_equivalent,0.004989,0.096641,0.212934,0.818734,0.020775,1.000000,0.589099,-0.005969,-0.000402,-0.292364,0.579060
bet_ratio,0.004106,-0.002174,0.026245,0.681155,-0.663164,0.589099,1.000000,-0.008840,-0.005798,-0.364398,0.978102
home_crowd,-0.000007,0.003055,0.004817,-0.007454,0.002475,-0.005969,-0.008840,1.000000,0.769180,0.004288,-0.007215
total_crowd,0.000202,0.000224,0.005992,-0.001393,0.005301,-0.000402,-0.005798,0.769180,1.000000,0.004798,-0.004878
game_result,0.022822,-0.011366,-0.021292,-0.269000,0.236265,-0.292364,-0.364398,0.004288,0.004798,1.000000,-0.367574


In [3277]:
y = train["game_result"]
X = train.drop(["game_result"],axis=1)

# scaler=MinMaxScaler()
# X = scaler.fit_transform(X)

In [3278]:
from sklearn.model_selection import train_test_split 

x_train , x_test, y_train, y_test = train_test_split(X,y,test_size=0.00002,random_state=42)

In [3279]:
# scaler=StandardScaler()
# X_train = scaler.fit_transform(x_train)
# X_test = scaler.transform(x_test)
X_train = x_train
X_test = x_test

In [3280]:
# from sklearn.ensemble import RandomForestClassifier
# lr = RandomForestClassifier(bootstrap=True, n_estimators= 100)

# lr.fit(X_train,y_train)

# from sklearn.neighbors import KNeighborsClassifier
# lr = KNeighborsClassifier(n_neighbors=5)
# lr.fit(X_train, y_train)

# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression()
# lr.fit(X_train, y_train)

# from sklearn import svm
# lr = svm.SVC(kernel='linear') 
# lr.fit(X_train, y_train)

lr = xgboost.XGBClassifier(random_state = 1, max_depth = 15,n_estimators = 1000, min_samples_split = 2, min_samples_leaf = 1)
lr=lr.fit(X_train, y_train)

In [3281]:
# feature_importances = pd.DataFrame(lr.feature_importances_,
#                                    index = X_train.columns,
#                                     columns=['importance']).sort_values('importance',ascending=False)
# feature_importances

In [3282]:
run_gs = False

if run_gs:
    parameter_grid = {
                 'max_depth' : [4, 6, 8],
                 'n_estimators': [50, 10, 20],
                 'max_features': ['sqrt', 'auto', 'log2'],
                 'min_samples_split': [2, 3, 10],
                 'min_samples_leaf': [1, 3, 10],
                 'bootstrap': [True, False],
                 }
    forest = RandomForestClassifier()
    cross_validation = StratifiedKFold(n_splits=1)

    grid_search = GridSearchCV(forest,
                               scoring='accuracy',
                               param_grid=parameter_grid,
                               cv=cross_validation,
                               verbose=1,
                               n_jobs=-1
                              )

    grid_search.fit(X_train, y_train)
    model = grid_search
    parameters = grid_search.best_params_

    print('Best score: {}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    


In [3283]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error,r2_score,accuracy_score,confusion_matrix
y_pred= lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
acc= accuracy_score(y_test,y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
lmse = mean_squared_log_error(y_test, y_pred)
rmse = np.sqrt(mse)
lrmse = np.sqrt(lmse)
print("R2 score:", r2)
print("ACC:", acc)
print("MAE:", mae)
print("MSE:", mse)
print("LMSE:", lmse)
print("RMSE:", rmse)
print("LRMSE:", lrmse)

R2 score: nan
ACC: 1.0
MAE: 0.0
MSE: 0.0
LMSE: 0.0
RMSE: 0.0
LRMSE: 0.0


/home/freakcap/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [3284]:
from datetime import datetime
test['date'] = pd.to_datetime(test['date'],format='%m/%d/%Y')
# test['month'] = test['date'].dt.month
# test['dayofweek'] = test['date'].dt.dayofweek
# test['year'] = test['date'].dt.year
# test['day'] = test['date'].dt.day
# test['dayofyear'] = test['date'].dt.dayofyear
# test['weekofyear'] = test['date'].dt.weekofyear

test.drop(['date'], axis=1, inplace=True) 

In [3285]:
# test = pd.concat([test,pd.get_dummies(test['team_id'], prefix='team_id',dummy_na=False)],axis=1)
# test = pd.concat([test,pd.get_dummies(test['opp_team_id'], prefix='opp_team_id',dummy_na=False)],axis=1)

In [3286]:
test['team_id'] = test['team_id'].str.extract('(\\d+)', expand=True)
test['opp_team_id'] = test['opp_team_id'].str.extract('(\\d+)', expand=True)

test['team_id']=test['team_id'].astype(int)
test['opp_team_id']=test['opp_team_id'].astype(int)

In [3287]:
# test['Elo'].fillna(test.groupby('team_id')['Elo'].transform('mean'), inplace=True)
# test['opp_Elo'].fillna(test.groupby('opp_team_id')['opp_Elo'].transform('mean'), inplace=True)
# test['win_equivalent'].fillna(test.groupby('season_end')['win_equivalent'].transform('mean'), inplace=True)

test["temp"] = test["team_id"].map(str) + test["season_end"].map(str)
test["temp1"] = test["opp_team_id"].map(str) + test["season_end"].map(str)

test['Elo'].fillna(test.groupby('temp')['Elo'].transform('median'), inplace=True)
test['opp_Elo'].fillna(test.groupby('temp1')['opp_Elo'].transform('median'), inplace=True)
test['win_equivalent'].fillna(test.groupby('season_end')['win_equivalent'].transform('median'), inplace=True)

test['Elo'].fillna(test.groupby('team_id')['Elo'].transform('mean'), inplace=True)
test['opp_Elo'].fillna(test.groupby('opp_team_id')['opp_Elo'].transform('mean'), inplace=True)

test = test.dropna(axis = 0, how ='any')

test.drop(['temp'], axis=1, inplace=True) 
test.drop(['temp1'], axis=1, inplace=True) 

In [3288]:
# for index,row in test.iterrows():
    
#     if(test.loc[index,'bet_ratio']<0.5):
#         test.at[index,'Bet']=-1 
#     if(test.loc[index,'bet_ratio']>0.8):
#         test.at[index,'Bet']=1
#     if(test.loc[index,'bet_ratio']>=0.5 and test.loc[index,'bet_ratio']<=0.8):
#         test.at[index,'Bet']=0
#     if(test.loc[index,'Elo']<1350):
#         test.at[index,'elo1']=-1 
#     if(test.loc[index,'Elo']>1645):
#         test.at[index,'elo1']=1
#     if(test.loc[index,'Elo']>=1350 and test.loc[index,'Elo']<=1645):
#         test.at[index,'elo1']=0
#     if(test.loc[index,'opp_Elo']<1355):
#         test.at[index,'elo2']=-1 
#     if(test.loc[index,'opp_Elo']>1660):
#         test.at[index,'elo2']=1
#     if(test.loc[index,'opp_Elo']>=1355 and test.loc[index,'opp_Elo']<=1660):
#         test.at[index,'elo2']=0


In [3289]:
test['diff']=test.Elo-test.opp_Elo


In [3290]:
Id = test['Id']

test.drop(['Id'], axis=1, inplace=True)
test.drop(['game_seq'], axis=1, inplace=True)  
# test.drop(['season_end'], axis=1, inplace=True)        
# test.drop(['season_game_seq'], axis=1, inplace=True) 
# test.drop(['playoff'], axis=1, inplace=True)
test.drop(['team_id'], axis=1, inplace=True) 
# test.drop(['Elo'], axis=1, inplace=True)
test.drop(['opp_team_id'], axis=1, inplace=True)
# test.drop(['opp_Elo'], axis=1, inplace=True) 
# test.drop(['win_equivalent'], axis=1, inplace=True) 
# test.drop(['bet_ratio'], axis=1, inplace=True) 
# test.drop(['home_crowd'], axis=1, inplace=True) 
test.drop(['opp_crowd'], axis=1, inplace=True) 
# test.drop(['total_crowd'], axis=1, inplace=True) 

In [3291]:
labelencoder = LabelEncoder()
test['playoff'] = labelencoder.fit_transform(test['playoff'])
# test['team_id'] = labelencoder.fit_transform(test['team_id'])
# test['opp_team_id'] = labelencoder.fit_transform(test['opp_team_id'])

In [3292]:
test.head()

,season_end,season_game_seq,playoff,Elo,opp_Elo,win_equivalent,bet_ratio,home_crowd,total_crowd,diff
0,2006,45,0,1452.0272,1467.3956,36.577610,0.619442,8700,12000,-15.3684
1,2015,60,0,1498.2162,1283.8668,40.762897,0.859306,7000,9600,214.3494
2,2008,28,0,1495.3258,1392.2981,40.791542,0.762914,3800,5300,103.0277
3,1984,18,0,1602.1180,1528.8450,52.006989,0.730555,7000,7300,73.2730
4,1977,73,0,1481.2228,1571.5261,41.606894,0.513951,4900,5900,-90.3033


In [3293]:
# scaler=StandardScaler()
# test = scaler.transform(test)

In [3294]:
prediction_test=lr.predict(test)

In [3295]:
prediction_test=prediction_test.astype(int)

In [3296]:
my_solution=pd.DataFrame(prediction_test,Id,columns=["game_result"])
print(my_solution.shape)

(13107, 1)


In [3297]:
my_solution.to_csv("sub123.csv",index_label=["Id"])

In [3298]:
my_solution['game_result'].value_counts()

0    8658
1    4449
Name: game_result, dtype: int64